In [1]:
#https://argmining-org.github.io/2023/

from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.nn import Dropout
from torch.optim import SGD
from torch.nn import BCELoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_
import  numpy as np
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from tqdm.auto import tqdm
import random
import torch
from PIL import Image
from transformers import AdamW
#from coremodel2 import CoreModel
from torch.utils.data.sampler import SubsetRandomSampler

#import torchvision.transforms as transforms


/Users/vinuthanarayanamurthy/anaconda3/envs/mps_suppport/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
from os import listdir
df_global_train = pd.read_csv('data/ab_train_integrated2.csv')
df_global_train['image_tensor'] = ''

df_global_dev = pd.read_csv('data/ab_dev_integrated2.csv')
df_global_dev['image_tensor'] = ''

In [13]:
def load_model (model_name):
    base_model = AutoModel.from_pretrained("bert-base-uncased")
    #tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    return base_model

In [14]:
class StModel(torch.nn.Module):
        def __init__ (self) :
                super().__init__()
                self.model_name = 'bert-base-uncased'
                self.embed_dim = 768
                self.load_layers()

        def load_layers(self) :
                self.model1 = load_model(self.model_name)
                self.model2 = load_model(self.model_name)
                self.linear1 = Linear((2*self.embed_dim)+512,1000)
                self.linear2 = Linear(1000,self.embed_dim)
                self.linear3 = Linear(self.embed_dim,500)
                self.linear4= Linear(500,250)
                self.linear5= Linear(250,100)
                self.linear6 = Linear(100,1)
                self.linear7 = Linear(1,1)

                self_linear_image = Linear(256,1000)
                self.activation1 = ReLU()
                self.activation2 = ReLU()
                self.activation3 = ReLU()
                self.activation4 = ReLU()
                self.activation5 = ReLU()
                self.activation6 = ReLU()
                self.Sigmoid1 =  Sigmoid()
                self.dropout1 = Dropout(p=0.2)
                self.dropout2 = Dropout(p=0.2)
                self.dropout3 = Dropout(p=0.2)
                self.dropout4= Dropout(p=0.2)
                self.dropout5= Dropout(p=0.2)
                self.dropout6= Dropout(p=0.2)

        def mean_pooling(self,model_output,attention_mask) :
                token_embeddings = model_output[0]
                input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
                return torch.sum(token_embeddings*input_mask_expanded,1)/torch.clamp(input_mask_expanded.sum(1),min=1e-9)
        
        def forward (self,tweettext,imagetext,imagetensor):
                emb1 = self.model1(**tweettext)
                m1= self.mean_pooling(emb1,tweettext['attention_mask'])
                
                emb2 = self.model2(**imagetext)
                m2= self.mean_pooling(emb2,imagetext['attention_mask'])
                
                emb3= imagetensor
                
                merge= torch.cat((m1,m2,emb3),1)
                
                x= self.dropout1(merge)
                x=self.activation1(self.linear1(merge))
                x=self.dropout2(x)
                x=self.activation2(self.linear2(x))
                x=self.dropout3(x)
                x=self.activation2(self.linear3(x))
                x=self.dropout4(x)
                x=self.activation4(self.linear4(x))
                x=self.dropout5(x)
                x=self.activation5(self.linear5(x))
                x=self.dropout6(x)
                x=self.activation6(self.linear6(x))
                x=self.Sigmoid1(self.linear7(x))
                 

                return x



In [19]:
#data loader standard code 
#on lines of kaggle Custom dataset/dataloader to load BERT tokenizer
class MyDataset_train(Dataset):
    
    def __init__(self, ab_path_url ):
        print (ab_path_url,flush=True)
        self.tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
        
        self.ab_data = pd.read_csv(ab_path_url)
        
        
    def __getitem__(self, index):
        
        #print('getitem1',index)
        if (self.ab_data['stance'][index] == 'support'):
            ab_stance = 0
        else :
            ab_stance = 1
        #print('getitem2',index)
        sample = {
             "ab_tweet_text": self.tokenizer(self.ab_data['tweet_text'][index],max_length=64,padding='max_length',truncation=True,return_tensors = 'pt'),
             "ab_image_text": self.tokenizer(self.ab_data['image_text'][index],max_length=64,padding='max_length',truncation=True,return_tensors = 'pt'),
             "ab_image_tensor" : df_global_train['image_tensor'][index],
             "labels_st" : torch.tensor(ab_stance)
           
        }
        #print('getitem3',index)
        #print('sample',sample)
        #print('getitem4',index)
        return sample
    
    def __len__(self):
        #print("get_item_length")
        return len(self.ab_data)

In [20]:
#data loader standard code 
#on lines of kaggle Custom dataset/dataloader to load BERT tokenizer
class MyDataset_dev(Dataset):
    
    def __init__(self, ab_path_url ):
        print (ab_path_url,flush=True)
        self.tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
        
        self.ab_data = pd.read_csv(ab_path_url)
        
        
    def __getitem__(self, index):
        
        #print('getitem1',index)
        if (self.ab_data['stance'][index] == 'support'):
            ab_stance = 0
        else :
            ab_stance = 1
        #print('getitem2',index)
        sample = {
             "ab_tweet_text": self.tokenizer(self.ab_data['tweet_text'][index],max_length=64,padding='max_length',truncation=True,return_tensors = 'pt'),
             "ab_image_text": self.tokenizer(self.ab_data['image_text'][index],max_length=64,padding='max_length',truncation=True,return_tensors = 'pt'),
             "ab_image_tensor" : df_global_dev['image_tensor'][index],
             "labels_st" : torch.tensor(ab_stance)
           
        }
        #print('getitem3',index)
        #print('sample',sample)
        #print('getitem4',index)
        return sample
    
    def __len__(self):
        #print("get_item_length")
        return len(self.ab_data)

In [21]:
# collate data in batch 
def collate(batch):

    #print("function collate")
    features = {}
     
    
    ab_tweet_ids = [item['ab_tweet_text']['input_ids'][0]for item in batch]
    ab_tweet_ttypes = [item['ab_tweet_text']['token_type_ids'][0]for item in batch]
    ab_tweet_attn = [item['ab_tweet_text']['attention_mask'][0]for item in batch]

    #ab_tweet_ids = [torch.tensor(item['ab_tweet_text']['input_ids'])[0]for item in batch]
    #ab_tweet_ttypes = [torch.tensor(item['ab_tweet_text']['token_type_ids'])[0]for item in batch]
    #ab_tweet_attn = [torch.tensor(item['ab_tweet_text']['attention_mask'])[0]for item in batch]


    
    ab_image_text_ids = [item['ab_image_text']['input_ids'][0]for item in batch]
    ab_image_text_ttypes = [item['ab_image_text']['token_type_ids'][0] for item in batch]
    ab_image_text_attn = [item['ab_image_text']['attention_mask'][0] for item in batch]

   

    ab_image_tensor = [item['ab_image_tensor'] for item in batch]

    labels_st_val = [item['labels_st'] for item in batch]

    
    
    features['ab_tweet'] = {
        'input_ids' : torch.stack((ab_tweet_ids)),
        'attention_mask' :torch.stack((ab_tweet_attn))
    }

    
    features['ab_image_text'] = {
      'input_ids' : torch.stack(ab_image_text_ids),
      'attention_mask' :torch.stack(ab_image_text_attn)
    }
   
    #print (ab_image_tensor,flush=True)
    features['ab_image_tensor']= torch.tensor(ab_image_tensor,dtype=torch.float32)
    
   
    
    #print('labels_st_val',labels_st_val)
    features['label_st']= torch.tensor(labels_st_val,dtype=torch.float32)
    
    return features

In [22]:
def train_stance_actual():
    device = torch.device('mps')
    print(device)
    ab_path_train= 'data/ab_train_integrated2.csv'
    ab_dataset_train =  MyDataset_train(ab_path_train)
    ab_dataset_train_size = len(ab_dataset_train)
    
    #validation_split = 0.05
    #split = int(np.floor(validation_split * ab_dataset_train_size))
    indices = list(range(ab_dataset_train_size))
    #train_indices,val_indices = indices[split:], indices[:split]
    train_indices = indices

    train_sampler = SubsetRandomSampler(train_indices)
   
    batch_size =  5#5
    learning_rate= 0.000005
    
    train_dataloader =  DataLoader(ab_dataset_train,batch_size, sampler=train_sampler, collate_fn= collate, shuffle=False)


    ab_path_dev = 'data/ab_dev_integrated2.csv'
    ab_dataset_dev =  MyDataset_dev(ab_path_dev)
    ab_dataset_dev_size = len(ab_dataset_dev)
    
    #validation_split = 0.05
    #split = int(np.floor(validation_split * ab_dataset_train_size))
    indices = list(range(ab_dataset_dev_size))
    #train_indices,val_indices = indices[split:], indices[:split]
    val_indices = indices

     
    val_sampler = SubsetRandomSampler(val_indices)

    

    val_dataloader = DataLoader(ab_dataset_dev,batch_size, sampler= val_sampler ,collate_fn= collate, shuffle=False)
    
    
    #criterion = torch.nn.CrossEntropyLoss()
    criterion = torch.nn.BCEWithLogitsLoss()
    st_model = StModel()
    st_model.to(device)
    st_model.float()
    optimizer = AdamW(st_model.parameters(),learning_rate)
    epochs = 10  #15
    #for batch in tqdm(train_dataloader):
    i=0
    for epoch in range(0,epochs):
        print("epoch",epoch)
        train_loss_per_batch= []
        train_running_loss=[]
        val_loss_per_batch=[]
        train_pred = []
        val_pred=[]
        val_gt = []
        train_stance =[]
        val_stance =[]

        st_model.train()


        for batch in tqdm(train_dataloader):
            #print('1')
            #ab_tweet.to(device), ab_image_text.to(device), label_st.to(device)
            ab_tweet = batch['ab_tweet']
            ab_image_text = batch['ab_image_text']
            #ab_image_tensor = batch['ab_image_tensor']
             #label_st= batch['label_st']
            #ab_image_tensor = batch[]
            #print('2')
            ab_tweet_input ={k:v.to(device) for k,v in ab_tweet.items()}
            ab_image_input ={k:v.to(device) for k,v in ab_image_text.items()}
            ab_image_tensor =batch['ab_image_tensor'].float().to(device)
            label_st_input = batch['label_st'].float().to(device)
            #print('3')
            st_pred = st_model(ab_tweet_input,ab_image_input,ab_image_tensor)

            #print (st_pred.shape)
            #print(label_st_input.shape)
            st_loss = criterion(st_pred.squeeze(-1), label_st_input)

            train_loss = st_loss
            print ("train_loss",train_loss)

            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()

            #train_loss_per_batch.append(train_loss.item())

        st_model.eval()
        
        with torch.no_grad():
         for batch in tqdm(val_dataloader):

            #ab_tweet.to(device), ab_image_text.to(device), label_st.to(device)
            ab_tweet = batch['ab_tweet']
            ab_image_text = batch['ab_image_text']
             #label_st= batch['label_st']

            ab_tweet_input ={k:v.to(device) for k,v in ab_tweet.items()}
            ab_image_input ={k:v.to(device) for k,v in ab_image_text.items()}
            ab_image_tensor =batch['ab_image_tensor'].float().to(device)
            label_st_input = batch['label_st'].float().to(device)

            st_pred = st_model(ab_tweet_input,ab_image_input,ab_image_tensor)

            #print (st_pred.shape)
            #print(label_st_input.shape)
            val_st_loss = criterion(st_pred.squeeze(-1), label_st_input)

            val_loss = val_st_loss
            print ("val_loss",val_loss)

            

            #val_loss_per_batch.append(val_loss.item())

    


        #train_running_loss = sum(train_loss_per_batch)/len(train_loss_per_batch)*batch_size
        #train_loss_dict[epoch] = [train_running_loss, sum(train_loss_per_batch)/len(train_loss_per_batch)]





In [23]:
train_stance_actual()

mps
data/ab_train_integrated2.csv
data/ab_dev_integrated2.csv


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predic

train_loss tensor(0.6121, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


  1%|          | 1/177 [00:00<02:54,  1.01it/s]

train_loss tensor(0.6121, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


...

val_loss tensor(0.5224, device='mps:0')
val_loss tensor(0.3503, device='mps:0')
val_loss tensor(0.3136, device='mps:0')
val_loss tensor(0.4213, device='mps:0')
val_loss tensor(0.6202, device='mps:0')


 75%|███████▌  | 15/20 [00:00<00:00, 23.28it/s]

val_loss tensor(0.6087, device='mps:0')
val_loss tensor(0.4741, device='mps:0')
val_loss tensor(0.6203, device='mps:0')
val_loss tensor(0.8534, device='mps:0')
val_loss tensor(0.4746, device='mps:0')


100%|██████████| 20/20 [00:00<00:00, 23.20it/s]

val_loss tensor(0.4687, device='mps:0')
val_loss tensor(0.4281, device='mps:0')
val_loss tensor(0.5398, device='mps:0')
val_loss tensor(0.3144, device='mps:0')
val_loss tensor(0.3137, device='mps:0')


In [ ]:
https://shairozsohail.medium.com/exploring-deep-embeddings-fa677f0e7c90

In [3]:
from torchvision import datasets, transforms
import torch

data_dir = 'data_experiment'
# create dataloader with required transforms 
tc = transforms.Compose([
       transforms.Resize((256, 256)),
       transforms.ToTensor()              
   ])

image_datasets = datasets.ImageFolder(data_dir, transform=tc)
dloader = torch.utils.data.DataLoader(image_datasets, batch_size=10, shuffle=False)

print(image_datasets) # 984

Dataset ImageFolder
    Number of datapoints: 984
    Root location: data_experiment
    StandardTransform
Transform: Compose(
               Resize(size=(256, 256), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
           )


In [4]:
# fetch pretrained model
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

Using cache found in /Users/vinuthanarayanamurthy/.cache/torch/hub/pytorch_vision_v0.10.0
/Users/vinuthanarayanamurthy/anaconda3/envs/mps_suppport/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/vinuthanarayanamurthy/anaconda3/envs/mps_suppport/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
layer = model._modules.get('avgpool')

In [6]:
def copy_embeddings(m, i, o):
  """Copy embeddings from the penultimate layer.
    """
  #print("copy2")
  o = o[:, :, 0, 0].detach().numpy().tolist()
  outputs.append(o)

outputs = []
#print("copy1 ")
# attach hook to the penulimate layer
_ = layer.register_forward_hook(copy_embeddings)

In [7]:
model.eval() # Inference mode

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [8]:
for X, y in dloader:
  _ = model(X)

In [9]:
# flatten list of embeddings to remove batches
list_embeddings = [item for sublist in outputs for item in sublist]

#print(len(list_embeddings)) # returns 984
print(np.array(list_embeddings[0]).shape) #returns (512,)

(512,)


In [10]:



    
folder_dir = 'data_experiment/abortion'
tweet_images_length = df_global_train['tweet_id'].shape[0]
print(tweet_images_length)
for i in range(tweet_images_length):
    image_id = df_global_train['tweet_id'][i]
    image_id =  str(image_id)+ '.jpg'
    
    #print (image_id)
    image_index=0
    for images in os.listdir(folder_dir):
        #print ("images", images)
        if (images == image_id) :
            print(i,image_index)
            print(images)
            print(image_id)
            df_global_train['image_tensor'][i] = list_embeddings[image_index]
        image_index = image_index+1   


884
0 785
1148501065308004357.jpg
1148501065308004357.jpg
1 439
1103872992537276417.jpg
1103872992537276417.jpg
2 564
1151528583623585794.jpg
1151528583623585794.jpg
3 406
1100166844026109953.jpg
1100166844026109953.jpg
4 174
1021830413550067713.jpg
1021830413550067713.jpg
5 376
1101125627984392192.jpg
1101125627984392192.jpg
6 955
1021869652329213952.jpg
1021869652329213952.jpg
7 141
1151273204926881795.jpg
1151273204926881795.jpg
8 116
1022558421046976512.jpg
1022558421046976512.jpg
9 576
1141294560989143040.jpg
1141294560989143040.jpg
10 628
1145368553622843393.jpg
1145368553622843393.jpg
11 176
1041895328788574208.jpg
1041895328788574208.jpg
12 651
1143827609471655937.jpg
1143827609471655937.jpg
13 596
1101529254423130112.jpg
1101529254423130112.jpg
14 160
1101557513139511296.jpg
1101557513139511296.jpg
15 264
1151002554672779264.jpg
1151002554672779264.jpg
16 952
1100840703595171840.jpg
1100840703595171840.jpg
17 782
1098995846186704897.jpg
1098995846186704897.jpg
18 760
114398897

/var/folders/p5/hv9b28g55619pd8pxlh67ns00000gn/T/ipykernel_17063/1522458104.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_global_train['image_tensor'][i] = list_embeddings[image_index]


382 68
1183891722121687040.jpg
1183891722121687040.jpg
383 421
1052412036334972928.jpg
1052412036334972928.jpg
384 590
1074016436606033920.jpg
1074016436606033920.jpg
385 177
1268920037286404103.jpg
1268920037286404103.jpg
386 112
1233474999509495809.jpg
1233474999509495809.jpg
387 751
1186582969546366976.jpg
1186582969546366976.jpg
388 157
1236410105408692224.jpg
1236410105408692224.jpg
389 348
1048289435761221632.jpg
1048289435761221632.jpg
390 694
1261394696640663553.jpg
1261394696640663553.jpg
391 8
1303634457606447104.jpg
1303634457606447104.jpg
392 722
1053239145093447681.jpg
1053239145093447681.jpg
393 302
1308121875298553856.jpg
1308121875298553856.jpg
394 164
1163917236945465345.jpg
1163917236945465345.jpg
395 737
1363202047416414208.jpg
1363202047416414208.jpg
396 425
1071438731310063616.jpg
1071438731310063616.jpg
397 976
1161783373284724738.jpg
1161783373284724738.jpg
398 632
1227774390823260160.jpg
1227774390823260160.jpg
399 225
1245562974867251200.jpg
1245562974867251200

In [11]:
df_global_train


,Unnamed: 0,tweet_id,tweet_url,tweet_text,stance,persuasiveness,split,image_text,image_tensor
0,0,1148501065308004357,https://t.co/VQP1FHaWAg,Let's McGyver some Sanity in America!\n\nYou a...,support,no,train,REMEMBER THAT TIME | COULDNT GET MYSELF OUT OF...,"[0.8572938442230225, 0.5705414414405823, 0.189..."
1,1,1103872992537276417,https://t.co/zsyXYSeBkp,A child deserves a chance at life. A child des...,oppose,no,train,[ deserve to be loved. ACLJ.ORG,"[0.5855329036712646, 0.30495500564575195, 0.89..."
2,2,1151528583623585794,https://t.co/qSWvDX5MnM,"Dear prolifers: girls as young as 10, 11, 12 a...",support,no,train,12:21 amplivescience.com The results are horri...,"[0.692261278629303, 0.20225557684898376, 2.200..."
3,3,1100166844026109953,https://t.co/hxH8tFIHUu,The many States will attempt to amend their co...,support,no,train,KEEP ABORTION LEGAL SAFE md,"[0.34930357336997986, 0.9716804623603821, 0.32..."
4,4,1021830413550067713,https://t.co/5whvEEtoQR,"Every #abortion is wrong, no matter what metho...",oppose,yes,train,YOUR BODY? SINCE WHEN DOES A FEMALE BODY HAVE:...,"[0.905445396900177, 0.5552948713302612, 1.1869..."
...,...,...,...,...,...,...,...,...,...
879,879,1050027978875392001,https://t.co/XyAXBxSofW,"Today, in 1920, legendary jazz pianist and com...",oppose,no,train,You( OLY BE A LEGEnD IF BORN: MMMIMMM RADIANCE...,"[0.01616114377975464, 0.20322202146053314, 0.7..."
880,880,1181235194637910016,https://t.co/hvpeI3Rax7,Abortion is misogynistic. It puts mother again...,oppose,yes,train,Acivlisedsocietydoes not pttmother against chi...,"[0.0039016096852719784, 0.8583628535270691, 0...."
881,881,1194970305363623937,https://t.co/ZT3gzfrGxS,The vast majority of abortions are unnecessary...,oppose,yes,train,"""I've noticed that everyone who is for abortio...","[1.2217296361923218, 1.5377531051635742, 1.649..."
882,882,1237936411212382213,https://t.co/aOIjsQsQxl,"There are some persons who like abortion, even...",oppose,yes,train,dummy,"[0.6296133995056152, 0.16972514986991882, 1.12..."


In [12]:
folder_dir = 'data_experiment/abortion'
tweet_images_length = df_global_dev['tweet_id'].shape[0]
print(tweet_images_length)
for i in range(tweet_images_length):
    image_id = df_global_dev['tweet_id'][i]
    image_id =  str(image_id)+ '.jpg'
    
    #print (image_id)
    image_index=0
    for images in os.listdir(folder_dir):
        #print ("images", images)
        if (images == image_id) :
            print(i,image_index)
            print(images)
            print(image_id)
            df_global_dev['image_tensor'][i] = list_embeddings[image_index]
        image_index = image_index+1   

100
0 637
1081597919726190592.jpg
1081597919726190592.jpg
1 600
1213868223663419392.jpg
1213868223663419392.jpg
2 708
1040359568407183361.jpg
1040359568407183361.jpg
3 929
1030187200359682048.jpg
1030187200359682048.jpg
4 44
1269277873376034816.jpg
1269277873376034816.jpg
5 496
1176134142720917504.jpg
1176134142720917504.jpg
6 936
1123376795020156928.jpg
1123376795020156928.jpg
7 522
1244678862698496017.jpg
1244678862698496017.jpg
8 541
1162826213339074560.jpg
1162826213339074560.jpg
9 170
1360696515438776321.jpg
1360696515438776321.jpg
10 467
1167089522594734081.jpg
1167089522594734081.jpg
11 907
1051771637589106693.jpg
1051771637589106693.jpg
12 662
1299908001457790978.jpg
1299908001457790978.jpg
13 41
1079426932150743040.jpg
1079426932150743040.jpg
14 26
1172796352834822144.jpg
1172796352834822144.jpg
15 230
1074035476258934784.jpg
1074035476258934784.jpg
16 184
1068585957660680192.jpg
1068585957660680192.jpg
17 548
1053324488006361090.jpg
1053324488006361090.jpg
18 906
136531025572

/var/folders/p5/hv9b28g55619pd8pxlh67ns00000gn/T/ipykernel_17063/1120135149.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_global_dev['image_tensor'][i] = list_embeddings[image_index]
